In [1]:
#Implementing SCD Type 1

In [2]:
import pandas as pd
import sqlalchemy as sal
engine=sal.create_engine('mssql://DESKTOP-N6DEIMB\SQLEXPRESS/master?driver=ODBC+Driver+17+for+SQL+Server')
conn=engine.connect()

In [3]:
#implementing SCD Type1
def extract():
    df_products=pd.read_csv('products.txt')
    df_products_db=pd.read_sql_query('select * from product',con=conn)
    return df_products,df_products_db

def transform(df_products,df_products_db):
    df_merge=pd.merge(df_products,df_products_db,how='left',on='product_id')
    df_insert=df_merge[df_merge['product_name_y'].isna()]
    df_insert_final=df_insert.iloc[:,0:3]
    df_insert_final.columns=df_products_db.columns
    
    df_update=df_merge[df_merge['product_name_y'].notna()]
    df_update_final=df_update.iloc[:,0:3]
    df_update_final.columns=df_products_db.columns
    return df_insert_final,df_update_final

def load_staging(df_update_final):
    df_update_final.to_sql('products_stg',con=conn , index=False , if_exists = 'replace')
    conn.commit()

def update():
    query = sal.text("update product set price = products_stg.price, product_name=products_stg.product_name from products_stg where product.product_id=products_stg.product_id")
    p = conn.execute(query)
    conn.commit()
    
def load(df_insert_final):
    df_insert_final.to_sql('product',index=False,con=conn,if_exists='append')
    conn.commit()

In [11]:
df_products,df_products_db=extract()

In [12]:
df_insert_final,df_update_final=transform(df_products,df_products_db)

In [13]:
load(df_insert_final)

AttributeError: 'Connection' object has no attribute 'commit'

In [14]:
load_staging(df_update_final)

AttributeError: 'Connection' object has no attribute 'commit'

In [15]:
update()

AttributeError: 'Connection' object has no attribute 'commit'